In [22]:
import pandas as pd                 # for manipulating data
import pcdl                         # physicell data loader library
# import os, sys, subprocess
import math, os, sys, re
import os.path
import glob

In [3]:
################## Pathing Variables ########################

# relative path to input directory
# EDIT TO BE LOCATION OF ORIGINAL MODEL FILES
rel_input_dir = 'leukemia_output/'

# relative path to output directory
# EDIT TO BE LOCATION OF WHERE YOU WANT TO STORE THE NEW MODEL FILES
rel_output_dir = 'dataframes/'

########################### PATHING ###########################
# move to PhysiCell root directory - this assumes this script is one folder below the root
os.chdir("../")
full_path = os.getcwd()
print(full_path)

# INPUTS
input_dir = os.path.join(full_path, rel_input_dir)
print("Input directory: ")
print(input_dir)

# OUTPUTS
output_dir = os.path.join(full_path, rel_output_dir)
print("Output directory: ")
print(output_dir)

/Users/JohnMetzcar/Documents/GitHub/PhysiLab
Input directory: 
/Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/
Output directory: 
/Users/JohnMetzcar/Documents/GitHub/PhysiLab/dataframes/


In [33]:
# make sure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# list of intervention names
interventions = [f.name for f in os.scandir(input_dir) if f.is_dir()] 
interventions = sorted(interventions)
interventions

['EG_Ceramide_S1P_1_11',
 'EG_Ceramide_S1P_1_12',
 'EG_Ceramide_S1P_1_13',
 'EG_Ceramide_S1P_1_21',
 'EG_Ceramide_S1P_1_22',
 'EG_Ceramide_S1P_1_23',
 'EG_Ceramide_S1P_1_31',
 'EG_Ceramide_S1P_1_32',
 'EG_Ceramide_S1P_1_33',
 'EG_GAP_RAS_1_11',
 'EG_GAP_RAS_1_12',
 'EG_GAP_RAS_1_13',
 'EG_GAP_RAS_1_21',
 'EG_GAP_RAS_1_22',
 'EG_GAP_RAS_1_23',
 'EG_GAP_RAS_1_31',
 'EG_GAP_RAS_1_32',
 'EG_GAP_RAS_1_33',
 'EG_IL15_GAP_0_11',
 'EG_IL15_GAP_0_12',
 'EG_IL15_GAP_0_13',
 'EG_IL15_GAP_0_21',
 'EG_IL15_GAP_0_22',
 'EG_IL15_GAP_0_23',
 'EG_IL15_GAP_0_31',
 'EG_IL15_GAP_0_32',
 'EG_IL15_GAP_0_33',
 'EG_IL2RBT_IL2RB_0_11',
 'EG_IL2RBT_IL2RB_0_12',
 'EG_IL2RBT_IL2RB_0_13',
 'EG_IL2RBT_IL2RB_0_21',
 'EG_IL2RBT_IL2RB_0_22',
 'EG_IL2RBT_IL2RB_0_23',
 'EG_IL2RBT_IL2RB_0_31',
 'EG_IL2RBT_IL2RB_0_32',
 'EG_IL2RBT_IL2RB_0_33',
 'EG_IL2RB_MCL1_0_11',
 'EG_IL2RB_MCL1_0_12',
 'EG_IL2RB_MCL1_0_13',
 'EG_IL2RB_MCL1_0_21',
 'EG_IL2RB_MCL1_0_22',
 'EG_IL2RB_MCL1_0_23',
 'EG_IL2RB_MCL1_0_31',
 'EG_IL2RB_MCL1_0_32

In [34]:
# for each intervention
for i, intervention in enumerate(interventions):
    # create filepath
    path = input_dir + intervention

    # create a timeseries object
    mcds_ts = pcdl.TimeSeries(path)

    # create dictionary for this intervention
    # keys = time t; values = # live cells at time t
    data = {'intervention': interventions[i]}
    for mcds in mcds_ts.get_mcds_list():
        df_cell = mcds.get_cell_df()
        live_cells =len(df_cell[(df_cell['dead'] == False)])
        data[mcds.get_time()] = live_cells
    
    if (i == 0): df = pd.DataFrame([data])  # create the dataframe
    else: df.loc[len(df)] = data            # append the dictionary to the dataframe

# for root, dirs, files in interventions:
#     for name in files:
#         print(name)
#     for name in dirs:
#         print(name)

reading: /Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/EG_IL2RB_MCL1_0_11/PhysiCell_settings.xml
Warning @ pyMCDS._read_xml : cell_definition custom_data without variable type setting detected. ['drug_threshold', 'intervention_induced_apoptosis', 'proliferation_multiplier']
reading: /Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/EG_IL2RB_MCL1_0_11/output00000000.xml
working on metadata ...
working on mesh data ...
reading: /Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/EG_IL2RB_MCL1_0_11/initial_mesh0.mat
working on microenvironment data ...
reading: /Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/EG_IL2RB_MCL1_0_11/output00000000_microenvironment0.mat
parsing: anti_IL2RB_MCL1 data
working on discrete cell data ...
reading: /Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/EG_IL2RB_MCL1_0_11/output00000000_cells.mat
working on graph data ...
reading: /Users/JohnMetzcar/Documents/GitHub/PhysiLab/leukemia_output/EG_IL2RB_

In [36]:
df.to_csv(output_dir + 'live_cells.csv', index=False)  # save the dataframe to a csv file

In [37]:
pd.read_csv(output_dir + 'live_cells.csv')  # read the csv file to check that it saved correctly

,intervention
0,EG_Ceramide_S1P_1_11
1,EG_Ceramide_S1P_1_12
2,EG_Ceramide_S1P_1_13
3,EG_Ceramide_S1P_1_21
4,EG_Ceramide_S1P_1_22
...,...
274,SM_pro_TBETpro_Ceramidepro_ERK_22
275,SM_pro_TBETpro_Ceramidepro_ERK_23
276,SM_pro_TBETpro_Ceramidepro_ERK_31
277,SM_pro_TBETpro_Ceramidepro_ERK_32
